In [162]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
pd.set_option('display.max_columns', 25)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from scipy import interpolate

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

import seaborn as sns

**Crime Incident Report data: data cleaning and EDA**

In [3]:
crime = pd.read_csv('data/crime_incident_report.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426820 entries, 0 to 426819
Data columns (total 17 columns):
INCIDENT_NUMBER        426820 non-null object
OFFENSE_CODE           426820 non-null int64
OFFENSE_CODE_GROUP     426820 non-null object
OFFENSE_DESCRIPTION    426820 non-null object
DISTRICT               424650 non-null object
REPORTING_AREA         426820 non-null object
SHOOTING               1747 non-null object
OCCURRED_ON_DATE       426820 non-null object
YEAR                   426820 non-null int64
MONTH                  426820 non-null int64
DAY_OF_WEEK            426820 non-null object
HOUR                   426820 non-null int64
UCR_PART               426710 non-null object
STREET                 414430 non-null object
Lat                    399617 non-null float64
Long                   399617 non-null float64
Location               426820 non-null object
dtypes: float64(2), int64(4), object(11)
memory usage: 55.4+ MB


**Imputation**

For the latitude and longitude coordinate, we are going to impute the missing values by using the **average** values of the incidences which has the same **STREET** (hoping that there is some other incidents reported at the same street with non-null latitude/longitude coordinate). If the STREET data is also missing OR if there's no other observations in the dataset belonging to the same 'STREET' with non-null latitude and longitude, we're just going to drop the rows as there is no other appropriate info/fields that we can use to identify the location.


In [61]:
crime['Lat_imp'] = crime['Lat'].copy()
crime['Long_imp'] = crime['Long'].copy()

for i in range(len(crime)):    
    if np.isnan(crime['Lat'].loc[i]):
        st = crime['STREET'].loc[i]
        crime['Lat_imp'].loc[i] = np.mean(crime[crime['STREET']==st]['Lat'])
        crime['Long_imp'].loc[i] = np.mean(crime[crime['STREET']==st]['Long'])
    

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [82]:
crime = crime[crime['Lat_imp'].isna() == False]

In [85]:
crime[['Lat_imp', 'Long_imp']].isna().sum()

Lat_imp     0
Long_imp    0
dtype: int64

**Setting multi-class prediction column**

We're going to create a column for the crime type for the chosen 5 types:
* 0 indicates *larceny*
* 1 indicates *Burglary* (including *B&E* incidents)
* 2 indicates *Auto Theft*
* 3 indicates *M/V Accidents*
* 4 indicates *drugs*

Note that these are the umbrella categories, which includes all the subtypes below it (e.g. *M/V ACCIDENT* includes all instances of motor/vehicle accidents, such as *M/V ACCIDENT - PERSONAL INJURY, 'M/V ACCIDENT - PROPERTY  DAMAGE', 'M/V ACCIDENT - OTHER*, etc.)

We decided to just focus on these 5 crime types, and drop all other incidents from the dataset.

In [106]:
crime_type = np.zeros(len(crime))
for ind, offense in enumerate(crime['OFFENSE_DESCRIPTION']):
    if 'LARCENY' in offense:
        crime_type[ind] = 0
    elif ('B&E' in offense) or ('BURGLARY' in offense):
#       or  ('LARCENY' in offense)
        crime_type[ind] = 1
    elif 'AUTO THEFT' in offense:
        crime_type[ind] = 2
    elif 'M/V ACCIDENT' in offense:
        crime_type[ind] = 3
    elif 'DRUGS' in offense:
        crime_type[ind] = 4
    else:
        crime_type[ind] = 5

crime['type'] = crime_type


In [107]:
crime_filtered = crime[crime['type']<5].copy()
crime_filtered = crime_filtered.drop(['OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING','UCR_PART', 'STREET', 'Lat', 'Long','Location'], axis=1)

We're dropping the column *'SHOOTING'* because we don't have any other information that can be used to make predictions about this field to impute the missing values, and using the median/average value would not make sense. 

We're also dropping *'REPORTING_AREA', 'UCR_PART'* and *'DISTRICT'* because these fields does not really tell much (or have strong correlation) with the prediction for type of crime.

**Remove duplicates**

We may have duplicated rows because the same incidents might below to multiple types *(or subtypes)* of crime. 

If the types of crime are different (e.g. an incident which involve both drugs and M/V accidents), then we're going to keep that as 2 separate rows. 

However, if the incident has the same type (e.g. M/V accident) but involves more than 1 *subtypes* (e.g. the M/V accident caused both *property damage* AND *personal injury*), these rows will have the same numerical 'type' that we assigned previously. Thus, we're going to drop one of these rows to remove any duplicates in the filtered dataset.

In [108]:
crime_filtered = crime_filtered.drop_duplicates(subset = ['INCIDENT_NUMBER', 'type'])

In [109]:
crime_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101347 entries, 3 to 426814
Data columns (total 9 columns):
INCIDENT_NUMBER     101347 non-null object
OCCURRED_ON_DATE    101347 non-null object
YEAR                101347 non-null int64
MONTH               101347 non-null int64
DAY_OF_WEEK         101347 non-null object
HOUR                101347 non-null int64
type                101347 non-null float64
Lat_imp             101347 non-null float64
Long_imp            101347 non-null float64
dtypes: float64(3), int64(3), object(3)
memory usage: 7.7+ MB


In [110]:
crime_filtered['type'].value_counts()

0.0    47300
3.0    23270
4.0    15942
1.0     8920
2.0     5915
Name: type, dtype: int64

**Segment the map based on zipcode**

Using the raw data that map the latitude and longitude coordinate from https://gist.github.com/erichurst/7882666#file-us-zip-codes-from-2013-government-data, we're going to estimate the zipcode based on the shortest euclidian distance of the incident's coordinates to the center of each zipcode location in the raw data.

Note that visually, the zipcode might looked weird because it may contain less than 5 digits. This is because the field is intepreted as numeric values and hence the '0' in front will disappear. However, for our purpose in building a model, this is not going to be a problem as they still encode the same information as they are with the extra '0' digit in front.

In [111]:
zipcode = pd.read_csv('data/zipcode.csv')
zipcode.head()

,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


In [119]:
# this is like using knn (with n_neighbor=1), where the prediction y-values is the zipcode ('ZIP') and the predictors are the latitude and longitude
X = zipcode[['LAT', 'LNG']]
y = zipcode['ZIP']
knn1 = KNeighborsClassifier(n_neighbors=1).fit(X,y)

In [139]:
crime_filtered['zip_predicted'] = knn1.predict(crime_filtered[['Lat_imp','Long_imp']])

In [140]:
crime_filtered.head()

,INCIDENT_NUMBER,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,type,Lat_imp,Long_imp,zip_predicted
3,I192078642,2019-09-29 05:50:00,2019,9,Sunday,5,3.0,42.332419,-71.075013,2118
8,I192078636,2019-09-29 04:40:00,2019,9,Sunday,4,1.0,42.314630,-71.092615,2121
14,I192078623,2019-09-28 22:40:00,2019,9,Saturday,22,3.0,42.286065,-71.070010,2124
15,I192078622,2019-09-29 03:04:00,2019,9,Sunday,3,0.0,42.340070,-71.052794,2111
18,I192078615,2019-09-29 02:25:00,2019,9,Sunday,2,3.0,42.382589,-71.033420,2150


**Zillow datasets: Median home value and rental price per sq ft**

In [158]:
home = pd.read_csv('data/Zillow_MedianValuePerSqft_AllHomes.csv')
rental = pd.read_csv('data/Zillow_MedianZriPerSqft_AllHomes.csv')

# filter the data to those belonging for Boston/Cambridge MA only
home_Boston = pd.concat([home[home['City']=='Boston'], home[home['City']=='Cambridge']])
rental_Boston = pd.concat([rental[rental['City']=='Boston'], rental[rental['City']=='Cambridge']])

In [160]:
# print(home_Boston['City'].value_counts())
# print(rental_Boston['City'].value_counts())

In [177]:
rental_Boston.head() 

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2010-11,2010-12,2011-01,2011-02,2011-03,...,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06
579,58649,2135,Boston,MA,Boston-Cambridge-Newton,Suffolk County,580,2.396,2.370,2.346,2.324,2.290,...,2.808,2.812,2.814,2.816,2.818,2.818,2.816,2.820,2.828,2.850,2.884,2.920
1235,58638,2124,Boston,MA,Boston-Cambridge-Newton,Suffolk County,1236,1.022,1.012,1.020,1.036,1.068,...,1.710,1.712,1.712,1.714,1.716,1.722,1.726,1.732,1.740,1.750,1.762,1.780
1559,58641,2127,Boston,MA,Boston-Cambridge-Newton,Suffolk County,1560,1.676,1.670,1.710,1.788,1.894,...,2.778,2.778,2.778,2.782,2.806,2.836,2.862,2.876,2.892,2.910,2.948,2.988
1676,58644,2130,Boston,MA,Boston-Cambridge-Newton,Suffolk County,1677,1.332,1.326,1.350,1.384,1.420,...,2.258,2.260,2.262,2.266,2.262,2.258,2.250,2.256,2.262,2.272,2.312,2.360
1859,58642,2128,Boston,MA,Boston-Cambridge-Newton,Suffolk County,1860,1.670,1.628,1.600,1.576,1.566,...,2.346,2.350,2.352,2.350,2.342,2.330,2.322,2.324,2.336,2.350,2.406,2.468


For the home and rental value, we're going to match the data based on the zipcode. If the zipcode of the incident does not match with any zipzode in the zillow datasets ('home_Boston' and 'rental_Boston'), we're just going to use the average home/rental value of the 2 closest zipzode as an estimate.



In [184]:
#making the model for interpolation: home value

X_int = home_Boston['RegionName'] #the field 'RegionName' means the zipzode of the area
y_int = home_Boston['2019-09'] # the field '2019-09' means the median home value per sqft, assessed in Sept '19 (latest data)

# extrapolation: for the areas with zipzode not within the range of the zipzodes in the zillow datasets, we pick the 
# median home value for the closest area/zipcode as the estimate
# that is, for the zipzode (when encoded as integer) lower than smallest zipcode from the zillow datasets, 
# we pick the home value associated with the area with the smallest zipzode
# and similarly for higher value zipcode
min_zipcode = home_Boston[home_Boston['RegionName']==min(home_Boston['RegionName'])]['2019-09']
max_zipcode = home_Boston[home_Boston['RegionName']==max(home_Boston['RegionName'])]['2019-09']

home_interpolate = interpolate.interp1d(X_int, y_int, kind='linear', bounds_error=False, fill_value=(min_zipcode, max_zipcode))

crime_filtered['home_value'] = home_interpolate(crime_filtered['zip_predicted'])

In [186]:
#making the model for interpolation: rental price

X_int = rental_Boston['RegionName'] #the field 'RegionName' means the zipzode of the area
y_int = rental_Boston['2019-06'] # the field '2019-06' means the median rental price per sqft, assessed in June '19 (latest data)

# extrapolation: for the areas with zipzode not within the range of the zipzodes in the zillow datasets, we pick the 
# rent price at the closest area/zipcode as the estimate. 
# i.e. for the zipzode (when encoded as integer) lower than smallest zipcode from the zillow datasets, 
# we pick the rent price associated with the area with the smallest zipzode
# and similarly for higher value zipcode
min_zipcode = rental_Boston[rental_Boston['RegionName']==min(rental_Boston['RegionName'])]['2019-06']
max_zipcode = rental_Boston[rental_Boston['RegionName']==max(rental_Boston['RegionName'])]['2019-06']

rent_interpolate = interpolate.interp1d(X_int, y_int, kind='linear', bounds_error=False, fill_value=(min_zipcode, max_zipcode))

crime_filtered['rent_price'] = rent_interpolate(crime_filtered['zip_predicted'])

In [188]:
crime_filtered.head()

,INCIDENT_NUMBER,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,type,Lat_imp,Long_imp,zip_predicted,home_value,rent_price
3,I192078642,2019-09-29 05:50:00,2019,9,Sunday,5,3.0,42.332419,-71.075013,2118,1001.000000,3.632000
8,I192078636,2019-09-29 04:40:00,2019,9,Sunday,4,1.0,42.314630,-71.092615,2121,248.000000,1.836000
14,I192078623,2019-09-28 22:40:00,2019,9,Saturday,22,3.0,42.286065,-71.070010,2124,322.000000,1.780000
15,I192078622,2019-09-29 03:04:00,2019,9,Sunday,3,0.0,42.340070,-71.052794,2111,990.000000,3.808000
18,I192078615,2019-09-29 02:25:00,2019,9,Sunday,2,3.0,42.382589,-71.033420,2150,894.058824,3.036706
